### 4개 글자를 가진 단어를 학습시킨다. 3글자만 주어지면 한글자를 추천한다. 단어 완성

In [0]:
import tensorflow as tf
import numpy as np

## 알파벳 예측

In [0]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
 'h', 'i', 'j', 'k', 'l', 'm', 'n',
 'o', 'p', 'q', 'r', 's', 't', 'u',
 'v', 'w', 'x', 'y', 'z']

In [0]:
# {'a':1, 'b':1, 'c':2, ...'z':25}
num_dic = {n:index for index, n in enumerate(char_arr)}
dic_len=len(num_dic)
#num_dic

In [5]:
dic = {}
for idx, items in enumerate(char_arr):
  dic[items] = idx
print(dic)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [0]:
## 입력은 앞 3단어 wor, woo, dee, div, col
## 출력은 맨 끝단어 d, d, p, e, ..., s, d
##
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool',
 'load', 'love', 'kiss', 'kind']

In [0]:
def make_batch(seq_data):
  input_batch = []   # 앞 3글자를 숫자로-> one Hot encoding
  target_batch = []  # 영문을 숫자로 
  
  for seq in seq_data:
    # word,   wood,  deep 
    # 'wor' ,     'woo',       'dee'
    input = [num_dic[n] for n in seq[:-1]]  # 단어중 앞 3글자를 숫자로 바꿔줌.
    # [22,14,17], [22, 14,14], [3,4,4]
  
    # word의 seq[-1]는 'd'
    # wood의 seq[-1]는 'd'
    # deep의 seq[-1]는 'p'
  
    target = num_dic[seq[-1]]   # 단어중 맨 끝글자를 숫자로
    # 3, 3, 15  
  
    # input은 3개의 리스트를 갖는 숫자값을 one-hot-encoding
    # if input is [0, 1, 2]:
    # [[ 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    # [ 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
    # [ 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
    input_batch.append(np.eye(dic_len)[input])
    
    # sparse_softmax_cross_entropy_with_logits 는
    # one-hot 인코딩을 사용하지 않으므로 index 를 그냥 넘겨주면 됩니다.
    target_batch.append(target)
 
  return input_batch, target_batch
  

In [0]:
### 신경망 옵션 설정
learning_rate = 0.01
n_hidden = 128
total_epoch = 30

# 타입 스텝 :[1,2,3] -> 3
n_step = 3
n_input = n_class= dic_len

In [0]:
### 신경망 모델을 구성
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [0]:
## 기본 셀 구성
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)


In [0]:
## Dropout 기법
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

In [0]:
## 여러개 셀을 조합
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

In [0]:
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)


In [17]:
print(outputs)
print(states)

Tensor("rnn/transpose_1:0", shape=(?, 3, 128), dtype=float32)
(LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 128) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_6:0' shape=(?, 128) dtype=float32>))


In [18]:
# 최종 결과는 one-hot 인코딩 형식으로 만듭니다
# 결과를 Y의 다음 형식과 바꿔야 하기 때문에
# Y : [batch_size, n_class]

# --> outputs의 형태 : [batch_size, n_step, n_hidden]
# --> transpose 순서 변경 [n_step, batch_size, n_hidden]
# outputs[-1]  # output의 마지막 데이터
# tf.transpose 함수를 이용하여 n_step과 batch_size의 차원의 순서를 바꾸고,
# n_step 차원을 제거하여 마지막 단계의 결과값을 취함.
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
print(outputs)

Tensor("strided_slice:0", shape=(?, 128), dtype=float32)


In [0]:
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
#########
# 신경망 모델 학습
######
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data)

In [21]:
for epoch in range(total_epoch):
  _, loss = sess.run([optimizer, cost],
                    feed_dict={X:input_batch, Y:target_batch})
  print('Epoch:', '%04d'% (epoch+1),'cost =', '{:.6f}'.format(loss))
  #print(sess.run(outputs))
print('최적화 완료')

Epoch: 0001 cost = 3.987795
Epoch: 0002 cost = 3.095344
Epoch: 0003 cost = 1.696384
Epoch: 0004 cost = 1.450113
Epoch: 0005 cost = 0.900348
Epoch: 0006 cost = 1.202294
Epoch: 0007 cost = 0.651282
Epoch: 0008 cost = 0.445989
Epoch: 0009 cost = 0.545697
Epoch: 0010 cost = 0.483621
Epoch: 0011 cost = 0.337010
Epoch: 0012 cost = 0.207434
Epoch: 0013 cost = 0.358402
Epoch: 0014 cost = 0.437751
Epoch: 0015 cost = 0.189798
Epoch: 0016 cost = 0.062782
Epoch: 0017 cost = 0.089701
Epoch: 0018 cost = 0.316631
Epoch: 0019 cost = 0.102637
Epoch: 0020 cost = 0.153231
Epoch: 0021 cost = 0.085638
Epoch: 0022 cost = 0.024461
Epoch: 0023 cost = 0.173081
Epoch: 0024 cost = 0.088767
Epoch: 0025 cost = 0.064352
Epoch: 0026 cost = 0.043822
Epoch: 0027 cost = 0.023276
Epoch: 0028 cost = 0.227618
Epoch: 0029 cost = 0.067253
Epoch: 0030 cost = 0.024401
최적화 완료


In [25]:
sess.run(outputs[0], feed_dict={X: input_batch, Y: target_batch})
# sess.run(states[1], feed_dict={X: input_batch, Y: target_batch})


array([ 0.02673897, -0.15832934,  0.03743213, -0.04144966,  0.24978817,
       -0.2876715 ,  0.21274556, -0.25855398, -0.3265326 , -0.23456919,
       -0.03790496, -0.17288154,  0.15795808,  0.11091242, -0.37911826,
        0.46414173,  0.13307692,  0.3327922 , -0.02480455,  0.30198112,
        0.2886491 ,  0.23800486, -0.3732837 , -0.11717228,  0.14842913,
        0.304662  , -0.24621211,  0.25341344, -0.18444057, -0.16714987,
        0.21552907, -0.0211653 ,  0.01985679, -0.03218716,  0.27688   ,
        0.2934438 , -0.3689568 ,  0.25812522,  0.17060809, -0.04600568,
        0.0584036 ,  0.16703029, -0.30099258,  0.14083838,  0.13083096,
       -0.09376228, -0.08692893,  0.17533007,  0.04957275,  0.03365602,
       -0.28174967, -0.231933  , -0.36497298,  0.18398014, -0.09110282,
       -0.23196267, -0.142268  , -0.13962758, -0.09853227, -0.18688701,
       -0.33317605, -0.25526878,  0.29426676, -0.22423495,  0.09197083,
       -0.31563538, -0.47124776,  0.12475473, -0.15642011, -0.01

In [0]:
#########
# 결과 확인
######
# 레이블값이 정수이므로 예측값도 정수로 변경해줍니다.
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
# one-hot 인코딩이 아니므로 입력값을 그대로 비교합니다.
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))
input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val = sess.run([prediction, accuracy],
 feed_dict={X: input_batch, Y: target_batch})

In [24]:
predict_words = []
for idx, val in enumerate(seq_data):
 last_char = char_arr[predict[idx]]
 predict_words.append(val[:3] + last_char)
print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)



=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0
